In [23]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dot, Activation, Concatenate
import numpy as np

In [24]:
english_sentences = [
    "i love apples",
    "i love bananas",
    "i like bananas",
    "you like bananas",
    "you love bananas",
    "she loves bananas",
    "she eats bread",
    "he drinks water",
    "we play football",
    "they read books",
    "i am a student",
    "you are my friend",
    "she is very happy",
    "he goes to school",
    "we watch movies",
    "they eat rice",
    "i like music",
    "you love your family",
    "she cooks food",
    "he writes a letter"
]
tamil_sentences = [
    " எனக்கு ஆப்பிள் பிடிக்கும் ",
    " எனக்கு வாழைப்பழம் பிடிக்கும் ",
    " எனக்கு வாழைப்பழம் பிடிக்கும் ",
    " உங்களுக்கு வாழைப்பழம் பிடிக்கும் ",
    " உங்களுக்கு வாழைப்பழம் பிடிக்கும் ",
    " அவளுக்கு வாழைப்பழம் பிடிக்கும் ",
    " அவள் ரொட்டி சாப்பிடுகிறாள் ",
    " அவன் தண்ணீர் குடிக்கிறான் ",
    " நாங்கள் கால்பந்து விளையாடுகிறோம் ",
    " அவர்கள் புத்தகங்கள் படிக்கிறார்கள் ",
    " நான் ஒரு மாணவன் ",
    " நீங்கள் என் நண்பர் ",
    " அவள் மிகவும் மகிழ்ச்சியாக இருக்கிறாள் ",
    " அவன் பள்ளிக்குச் செல்கிறான் ",
    " நாங்கள் திரைப்படங்கள் பார்க்கிறோம் ",
    " அவர்கள் சாதம் சாப்பிடுகிறார்கள் ",
    " எனக்கு இசை பிடிக்கும் ",
    " நீங்கள் உங்கள் குடும்பத்தை நேசிக்கிறீர்கள் ",
    " அவள் உணவு சமைக்கிறாள் ",
    " அவன் ஒரு கடிதம் எழுதுகிறான் "
]

In [25]:
src_tokenizer = Tokenizer(oov_token="")
src_tokenizer.fit_on_texts(english_sentences)
src_seq = src_tokenizer.texts_to_sequences(english_sentences)

tgt_tokenizer = Tokenizer(filters='', oov_token="")
tgt_tokenizer.fit_on_texts(tamil_sentences)
tgt_seq = tgt_tokenizer.texts_to_sequences(tamil_sentences)

src_vocab_size = len(src_tokenizer.word_index) + 1
tgt_vocab_size = len(tgt_tokenizer.word_index) + 1

In [26]:
max_src_len = max(len(s) for s in src_seq)
max_tgt_len = max(len(s) for s in tgt_seq)

encoder_input = pad_sequences(src_seq, maxlen=max_src_len, padding='post')

decoder_input = pad_sequences(
    [s[:-1] for s in tgt_seq],
    maxlen=max_tgt_len-1,
    padding='post'
)

decoder_target = pad_sequences(
    [s[1:] for s in tgt_seq],
    maxlen=max_tgt_len-1,
    padding='post'
)

In [27]:
embedding_dim = 32
latent_dim = 64

encoder_inputs = Input(shape=(max_src_len,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs)

encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_tgt_len-1,))
dec_emb = Embedding(tgt_vocab_size, embedding_dim)(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)


In [28]:
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)

context = Dot(axes=[2, 1])([attention, encoder_outputs])
decoder_combined = Concatenate(axis=-1)([context, decoder_outputs])


output = Dense(tgt_vocab_size, activation='softmax')(decoder_combined)

In [29]:
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


model.fit(
    [encoder_input, decoder_input],
    np.expand_dims(decoder_target, -1),
    batch_size=4,
    epochs=400,
    shuffle=False,
    verbose=1
)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 4, 32)     │      1,408 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, 3, 32)     │      1,376 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 4, 64),   │     24,832 │ embedding_4[0][0] │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, 3, 64),   │     24,832 │ embedding_5[0][0… │
│                     │ (None, 64),       │            │ lstm_4[0][1],     │
│                     │ (None, 64)]       │            │ lstm_4[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_4 (Dot)         │ (None, 3, 4)      │          0 │ lstm_5[0][0],     │
│                     │                   │            │ lstm_4[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 3, 4)      │          0 │ dot_4[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_5 (Dot)         │ (None, 3, 64)     │          0 │ activation_2[0][… │
│                     │                   │            │ lstm_4[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 3, 128)    │          0 │ dot_5[0][0],      │
│ (Concatenate)       │                   │            │ lstm_5[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 3, 43)     │      5,547 │ concatenate_2[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 57,995 (226.54 KB)

 Trainable params: 57,995 (226.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.0792 - loss: 3.7575   
Epoch 2/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3132 - loss: 3.7311
Epoch 3/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3132 - loss: 3.7030
Epoch 4/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3132 - loss: 3.6645 
Epoch 5/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3132 - loss: 3.6068 
Epoch 6/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3132 - loss: 3.5130
Epoch 7/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3132 - loss: 3.3488 
Epoch 8/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3132 - loss: 3.0475 
Epoch 9/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3132 - loss: 2.5782 
Epoch 10/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3132 - loss: 2.3457 
Epoch 11/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3132 - loss: 2.2532 
Epoch 12/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3132 - 

In [30]:
reverse_tgt_index = {v: k for k, v in tgt_tokenizer.word_index.items()}

def translate(sentence):
    seq = src_tokenizer.texts_to_sequences([sentence])
    seq = pad_sequences(seq, maxlen=max_src_len, padding='post')

    target_seq = np.zeros((1, max_tgt_len-1))
    target_seq[0, 0] = tgt_tokenizer.word_index['']

    decoded = []

    for i in range(max_tgt_len-1):
        preds = model.predict([seq, target_seq], verbose=0)
        word_id = np.argmax(preds[0, i])
        word = reverse_tgt_index.get(word_id, '')

        if word == '' or word == '':
            break

        decoded.append(word)
        target_seq[0, i+1] = word_id

    return ' '.join(decoded)

In [31]:
print("English :", "you like bananas")
print("Tamil   :", translate("you like bananas"))

print("English :", "i love apples")
print("Tamil   :", translate("i love apples"))

English : you like bananas
Tamil   : வாழைப்பழம் பிடிக்கும்
English : i love apples
Tamil   : ஆப்பிள் பிடிக்கும்
